In [ ]:
pip install requests beautifulsoup4 pandas

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def scrape_page(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')

  table = soup.find('table')

  rows = table.find_all('tr')
  movie_data = []

  for row in rows[1:]:
    cols = row.find_all('td')
    rank = cols[0].get_text(strip=True)
    title = cols[1].get_text(strip=True)
    worldwide_lifetime_gross = cols[2].get_text(strip=True)
    domestic_lifetime_gross = cols[3].get_text(strip=True)
    domestic_percentage = cols[4].get_text(strip=True)
    foreign_lifetime_gross = cols[5].get_text(strip=True)
    foreign_percentage = cols[6].get_text(strip=True)
    year = cols[7].get_text(strip=True)

    movie_data.append([rank, title, worldwide_lifetime_gross, domestic_lifetime_gross, domestic_percentage, foreign_lifetime_gross, foreign_percentage, year])

  return movie_data

all_movie_data = []
base_url = 'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/'

for offset in range(0, 1000, 200):
  url = f'{base_url}?offset={offset}'
  movie_data = scrape_page(url)
  all_movie_data.extend(movie_data)

df_top_lifetime_movie_gross = pd.DataFrame(all_movie_data, columns=['rank', 'title', 'worldwide_lifetime_gross', 'domestic_lifetime_gross', 'domestic_percentage', 'foreign_lifetime_gross', 'foreign_percentage', 'year'])
df_top_lifetime_movie_gross

,rank,title,worldwide_lifetime_gross,domestic_lifetime_gross,domestic_percentage,foreign_lifetime_gross,foreign_percentage,year
0,1,Avatar,"$2,923,706,026","$785,221,649",26.9%,"$2,138,484,377",73.1%,2009
1,2,Avengers: Endgame,"$2,799,439,100","$858,373,000",30.7%,"$1,941,066,100",69.3%,2019
2,3,Avatar: The Way of Water,"$2,320,250,281","$684,075,767",29.5%,"$1,636,174,514",70.5%,2022
3,4,Titanic,"$2,264,750,694","$674,292,608",29.8%,"$1,590,458,086",70.2%,1997
4,5,Star Wars: Episode VII - The Force Awakens,"$2,071,310,218","$936,662,225",45.2%,"$1,134,647,993",54.8%,2015
...,...,...,...,...,...,...,...,...
995,996,Snow White and the Seven Dwarfs,"$184,960,747","$184,925,486",100%,"$35,261",<0.1%,1937
996,997,Moulin Rouge!,"$184,935,252","$57,386,607",31%,"$127,548,645",69%,2001
997,998,Ode to My Father,"$184,827,559","$2,300,121",1.2%,"$182,527,438",98.8%,2014
998,999,Hop,"$184,367,665","$108,498,305",58.8%,"$75,869,360",41.2%,2011


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to scrape data for a specific year and week
def scrape_weekly_box_office(year, week):
    url = f"https://www.boxofficemojo.com/weekly/{year}W{str(week).zfill(2)}/"
    response = requests.get(url)

    # If request fails, return an empty list
    if response.status_code != 200:
        print(f"Failed to retrieve data for {year} week {week}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the table on the page
    table = soup.find('table')

    if table:
        rows = table.find_all('tr')
        weekly_data = []

        # Iterate over table rows, skipping the header row
        for row in rows[1:]:
            cols = row.find_all('td')
            if len(cols) > 1:
                rank = cols[0].get_text(strip=True)
                lw = cols[1].get_text(strip=True)
                release = cols[2].get_text(strip=True)
                gross = cols[3].get_text(strip=True)
                percent_lw = cols[4].get_text(strip=True)
                theaters = cols[5].get_text(strip=True)
                change = cols[6].get_text(strip=True)
                average = cols[7].get_text(strip=True)
                total_gross = cols[8].get_text(strip=True)
                weeks = cols[9].get_text(strip=True)
                distributor = cols[10].get_text(strip=True)

                weekly_data.append({
                    'Rank': rank,
                    'LW': lw,
                    'Release': release,
                    'Gross': gross,
                    '%± LW': percent_lw,
                    'Theaters': theaters,
                    'Change': change,
                    'Average': average,
                    'Total Gross': total_gross,
                    'Weeks': weeks,
                    'Distributor': distributor,
                    'Year': year,
                    'Week': week
                })
        return weekly_data
    return []


all_movie_data = []

# Loop through each year and each week
for year in range(1978, 2025):
    print(f'Scraping data for year: {year}')

    for week in range(1, 53):  # Scraping weeks 1 to 52
        try:
            weekly_data = scrape_weekly_box_office(year, week)
            if weekly_data:
                all_movie_data.extend(weekly_data)

        except Exception as e:
            print(f'Error scraping data for year {year}, week {week}: {e}')

# Convert the data to a DataFrame
df_movies = pd.DataFrame(all_movie_data)

# Reorder columns to match your desired output format
df_movies = df_movies[['Rank', 'LW', 'Release', 'Gross', '%± LW', 'Theaters', 'Change', 'Average', 'Total Gross', 'Weeks', 'Distributor', 'Year', 'Week']]

# Save to CSV
df_movies.to_csv('boxoffice_weekly_data_1978_2024.csv', index=False)

# Display the DataFrame
df_movies


Scraping data for year: 1978
Scraping data for year: 1979
Scraping data for year: 1980
Scraping data for year: 1981
Scraping data for year: 1982
Scraping data for year: 1983
Scraping data for year: 1984
Scraping data for year: 1985
Scraping data for year: 1986
Scraping data for year: 1987
Scraping data for year: 1988
Scraping data for year: 1989
Scraping data for year: 1990
Scraping data for year: 1991
Scraping data for year: 1992
Scraping data for year: 1993
Scraping data for year: 1994
Scraping data for year: 1995
Scraping data for year: 1996
Scraping data for year: 1997
Scraping data for year: 1998
Scraping data for year: 1999
Scraping data for year: 2000
Scraping data for year: 2001
Scraping data for year: 2002
Scraping data for year: 2003
Scraping data for year: 2004
Scraping data for year: 2005
Scraping data for year: 2006
Scraping data for year: 2007
Scraping data for year: 2008
Scraping data for year: 2009
Scraping data for year: 2010
Scraping data for year: 2011
Scraping data 

,Rank,LW,Release,Gross,%± LW,Theaters,Change,Average,Total Gross,Weeks,Distributor,Year,Week
0,1,-,Star Wars: Episode IV - A New Hope,"$792,809",-,107,-,"$7,409","$213,955,370",46,Twentieth Century Fox,1978,13
1,1,1,Star Wars: Episode IV - A New Hope,"$979,923",+23.6%,104,-3,"$9,422","$214,935,293",47,Twentieth Century Fox,1978,14
2,1,-,Damien: Omen II,"$5,861,992",-,525,-,"$11,165","$5,861,992",1,Twentieth Century Fox,1978,23
3,1,-,Grease,"$16,055,908",-,862,-,"$18,626","$16,055,908",1,Paramount Pictures,1978,24
4,1,-,Heaven Can Wait,"$540,313",-,-,-,-,"$540,313",1,Paramount Pictures,1978,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149559,58,59,Daddio,$941,-59%,2,-,$470,"$978,615",15,Sony Pictures Classics,2024,40
149560,59,52,Omni Loop,$865,-82%,6,-13,$144,"$40,269",3,Magnolia Pictures,2024,40
149561,60,47,The Featherweight,$509,-92%,2,-5,$254,"$22,011",3,mTuckman Media,2024,40
149562,61,69,Nostalghia2024 Re-release,$500,-,1,-,$500,"$261,294",34,Kino Lorber,2024,40


In [3]:
df_movies = pd.read_csv('boxoffice_weekly_data_1978_2024.csv')
df_movies['unique_id'] = df_movies['Release'] + '_' + df_movies['Year'].astype(str) + '_W' + df_movies['Week'].astype(str)
df_movies['Release'] = df_movies['Release'] + " (" + df_movies['Year'].astype(str) + ")"
df_movies

,Rank,LW,Release,Gross,%± LW,Theaters,Change,Average,Total Gross,Weeks,Distributor,Year,Week,unique_id
0,1,-,Star Wars: Episode IV - A New Hope (1978),"$792,809",-,107,-,"$7,409","$213,955,370",46,Twentieth Century Fox,1978,13,Star Wars: Episode IV - A New Hope_1978_W13
1,1,1,Star Wars: Episode IV - A New Hope (1978),"$979,923",+23.6%,104,-3,"$9,422","$214,935,293",47,Twentieth Century Fox,1978,14,Star Wars: Episode IV - A New Hope_1978_W14
2,1,-,Damien: Omen II (1978),"$5,861,992",-,525,-,"$11,165","$5,861,992",1,Twentieth Century Fox,1978,23,Damien: Omen II_1978_W23
3,1,-,Grease (1978),"$16,055,908",-,862,-,"$18,626","$16,055,908",1,Paramount Pictures,1978,24,Grease_1978_W24
4,1,-,Heaven Can Wait (1978),"$540,313",-,-,-,-,"$540,313",1,Paramount Pictures,1978,25,Heaven Can Wait_1978_W25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149559,58,59,Daddio (2024),$941,-59%,2,-,$470,"$978,615",15,Sony Pictures Classics,2024,40,Daddio_2024_W40
149560,59,52,Omni Loop (2024),$865,-82%,6,-13,$144,"$40,269",3,Magnolia Pictures,2024,40,Omni Loop_2024_W40
149561,60,47,The Featherweight (2024),$509,-92%,2,-5,$254,"$22,011",3,mTuckman Media,2024,40,The Featherweight_2024_W40
149562,61,69,Nostalghia2024 Re-release (2024),$500,-,1,-,$500,"$261,294",34,Kino Lorber,2024,40,Nostalghia2024 Re-release_2024_W40


In [4]:
df_movies = df_movies.rename(columns={
    'Rank': 'weekly_rank',
    'LW': 'rank_last_week',
    'Release': 'movie_title',
    'Gross': 'week_gross',
    '%± LW': 'gross_change',
    'Theaters': 'number_of_theaters',
    'Change': 'change_in_theatres',
    'Average': 'average_revenue_per_theatre',
    'Total Gross': 'current_total_gross',
    'Weeks' : 'release_week_number',
    'Distributor': 'distributor',
    'Year': 'year',
    'Week': 'week'
})

df_movies = df_movies[['unique_id', 'movie_title', 'weekly_rank', 'rank_last_week', 'week_gross', 'number_of_theaters', 'change_in_theatres', 'average_revenue_per_theatre', 'current_total_gross', 'release_week_number', 'distributor', 'year', 'week']]
df_movies

,unique_id,movie_title,weekly_rank,rank_last_week,week_gross,number_of_theaters,change_in_theatres,average_revenue_per_theatre,current_total_gross,release_week_number,distributor,year,week
0,Star Wars: Episode IV - A New Hope_1978_W13,Star Wars: Episode IV - A New Hope (1978),1,-,"$792,809",107,-,"$7,409","$213,955,370",46,Twentieth Century Fox,1978,13
1,Star Wars: Episode IV - A New Hope_1978_W14,Star Wars: Episode IV - A New Hope (1978),1,1,"$979,923",104,-3,"$9,422","$214,935,293",47,Twentieth Century Fox,1978,14
2,Damien: Omen II_1978_W23,Damien: Omen II (1978),1,-,"$5,861,992",525,-,"$11,165","$5,861,992",1,Twentieth Century Fox,1978,23
3,Grease_1978_W24,Grease (1978),1,-,"$16,055,908",862,-,"$18,626","$16,055,908",1,Paramount Pictures,1978,24
4,Heaven Can Wait_1978_W25,Heaven Can Wait (1978),1,-,"$540,313",-,-,-,"$540,313",1,Paramount Pictures,1978,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149559,Daddio_2024_W40,Daddio (2024),58,59,$941,2,-,$470,"$978,615",15,Sony Pictures Classics,2024,40
149560,Omni Loop_2024_W40,Omni Loop (2024),59,52,$865,6,-13,$144,"$40,269",3,Magnolia Pictures,2024,40
149561,The Featherweight_2024_W40,The Featherweight (2024),60,47,$509,2,-5,$254,"$22,011",3,mTuckman Media,2024,40
149562,Nostalghia2024 Re-release_2024_W40,Nostalghia2024 Re-release (2024),61,69,$500,1,-,$500,"$261,294",34,Kino Lorber,2024,40


In [5]:
from datetime import datetime, timedelta

df_movies['date'] = pd.to_datetime(df_movies['year'].astype(str) + df_movies['week'].astype(str) + '0', format='%Y%W%w')

df_movies

,unique_id,movie_title,weekly_rank,rank_last_week,week_gross,number_of_theaters,change_in_theatres,average_revenue_per_theatre,current_total_gross,release_week_number,distributor,year,week,date
0,Star Wars: Episode IV - A New Hope_1978_W13,Star Wars: Episode IV - A New Hope (1978),1,-,"$792,809",107,-,"$7,409","$213,955,370",46,Twentieth Century Fox,1978,13,1978-04-02
1,Star Wars: Episode IV - A New Hope_1978_W14,Star Wars: Episode IV - A New Hope (1978),1,1,"$979,923",104,-3,"$9,422","$214,935,293",47,Twentieth Century Fox,1978,14,1978-04-09
2,Damien: Omen II_1978_W23,Damien: Omen II (1978),1,-,"$5,861,992",525,-,"$11,165","$5,861,992",1,Twentieth Century Fox,1978,23,1978-06-11
3,Grease_1978_W24,Grease (1978),1,-,"$16,055,908",862,-,"$18,626","$16,055,908",1,Paramount Pictures,1978,24,1978-06-18
4,Heaven Can Wait_1978_W25,Heaven Can Wait (1978),1,-,"$540,313",-,-,-,"$540,313",1,Paramount Pictures,1978,25,1978-06-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149559,Daddio_2024_W40,Daddio (2024),58,59,$941,2,-,$470,"$978,615",15,Sony Pictures Classics,2024,40,2024-10-06
149560,Omni Loop_2024_W40,Omni Loop (2024),59,52,$865,6,-13,$144,"$40,269",3,Magnolia Pictures,2024,40,2024-10-06
149561,The Featherweight_2024_W40,The Featherweight (2024),60,47,$509,2,-5,$254,"$22,011",3,mTuckman Media,2024,40,2024-10-06
149562,Nostalghia2024 Re-release_2024_W40,Nostalghia2024 Re-release (2024),61,69,$500,1,-,$500,"$261,294",34,Kino Lorber,2024,40,2024-10-06


In [10]:
# Check for null values in the DataFrame
null_values = df_movies.isnull().sum()
print("Null Values per Column:\n", null_values)

# Check the data types of the columns
data_types = df_movies.dtypes
print("\nData Types per Column:\n", data_types)

Null Values per Column:
 unique_id                      0
movie_title                    0
weekly_rank                    0
rank_last_week                 0
week_gross                     0
number_of_theaters             0
change_in_theatres             0
average_revenue_per_theatre    0
current_total_gross            0
release_week_number            0
distributor                    0
date                           0
dtype: int64

Data Types per Column:
 unique_id                              object
movie_title                            object
weekly_rank                             int64
rank_last_week                          int64
week_gross                              int64
number_of_theaters                      int64
change_in_theatres                      int64
average_revenue_per_theatre             int64
current_total_gross                     int64
release_week_number                     int64
distributor                            object
date                           date

In [8]:
df_movies['distributor'] = df_movies['distributor'].fillna('')

df_movies['rank_last_week'] = df_movies['rank_last_week'].replace({'-': '0'}, regex=True).astype(int)

df_movies['week_gross'] = df_movies['week_gross'].replace({'-': '0'}, regex=True)
df_movies['week_gross'] = df_movies['week_gross'].replace({'\$': '', ',': ''}, regex=True).astype(int)

df_movies['number_of_theaters'] = df_movies['number_of_theaters'].replace({'-': '0', ',': ''}, regex=True).astype(int)


df_movies['average_revenue_per_theatre'] = df_movies['average_revenue_per_theatre'].replace({'-': '0'}, regex=True)
df_movies['average_revenue_per_theatre'] = df_movies['average_revenue_per_theatre'].replace({'\$': '', ',': ''}, regex=True).astype(int)

df_movies['current_total_gross'] = df_movies['current_total_gross'].replace({'-': '0'}, regex=True)
df_movies['current_total_gross'] = df_movies['current_total_gross'].replace({'\$': '', ',': ''}, regex=True).astype(int)

df_movies['release_week_number'] = df_movies['release_week_number'].replace({'-': '0'}, regex=True)
df_movies['release_week_number'] = df_movies['release_week_number'].replace({',': ''}, regex=True).astype(int)

In [9]:
def clean_change_in_theatres(value):
    if value == '-':
        return 0
    else:
        return value

df_movies['change_in_theatres'] = df_movies['change_in_theatres'].apply(clean_change_in_theatres)
df_movies['change_in_theatres'] = df_movies['change_in_theatres'].replace({',': ''}, regex=True).astype(int)

In [6]:
df_movies = df_movies.drop(columns=['year', 'week'])
df_movies

,unique_id,movie_title,weekly_rank,rank_last_week,week_gross,number_of_theaters,change_in_theatres,average_revenue_per_theatre,current_total_gross,release_week_number,distributor,date
0,Star Wars: Episode IV - A New Hope_1978_W13,Star Wars: Episode IV - A New Hope (1978),1,-,"$792,809",107,-,"$7,409","$213,955,370",46,Twentieth Century Fox,1978-04-02
1,Star Wars: Episode IV - A New Hope_1978_W14,Star Wars: Episode IV - A New Hope (1978),1,1,"$979,923",104,-3,"$9,422","$214,935,293",47,Twentieth Century Fox,1978-04-09
2,Damien: Omen II_1978_W23,Damien: Omen II (1978),1,-,"$5,861,992",525,-,"$11,165","$5,861,992",1,Twentieth Century Fox,1978-06-11
3,Grease_1978_W24,Grease (1978),1,-,"$16,055,908",862,-,"$18,626","$16,055,908",1,Paramount Pictures,1978-06-18
4,Heaven Can Wait_1978_W25,Heaven Can Wait (1978),1,-,"$540,313",-,-,-,"$540,313",1,Paramount Pictures,1978-06-25
...,...,...,...,...,...,...,...,...,...,...,...,...
149559,Daddio_2024_W40,Daddio (2024),58,59,$941,2,-,$470,"$978,615",15,Sony Pictures Classics,2024-10-06
149560,Omni Loop_2024_W40,Omni Loop (2024),59,52,$865,6,-13,$144,"$40,269",3,Magnolia Pictures,2024-10-06
149561,The Featherweight_2024_W40,The Featherweight (2024),60,47,$509,2,-5,$254,"$22,011",3,mTuckman Media,2024-10-06
149562,Nostalghia2024 Re-release_2024_W40,Nostalghia2024 Re-release (2024),61,69,$500,1,-,$500,"$261,294",34,Kino Lorber,2024-10-06


In [11]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

project_id = 'movie-genre-analysis-le-wagon'

Authenticated


In [ ]:
import pandas_gbq

dataset_name = 'Secondary_Data'
table_name = 'box_office_cleaned'

pandas_gbq.to_gbq(df_movies, f'{dataset_name}.{table_name}', project_id=project_id)

100%|██████████| 1/1 [00:00<00:00, 6442.86it/s]
